# Experimenting with Masked Data and Weighting in xarray

This notebook experiments with how weighting with masked data can affect the results of reduction operations such as simple averages and grouped averages. Specifically, this notebook looks into weighted temporal averaging.

In [1]:
# flake8: noqa f401
import numpy as np
import xarray as xr

import xcdat

## Create a sample monthly time series dataset

In [2]:
ds = xr.Dataset(
    coords={
        "lat": [-90],
        "lon": [0],
        "time": xr.DataArray(
            data=np.array(
                [
                    "2000-01-01T00:00:00.000000000",
                    "2000-02-01T00:00:00.000000000",
                    "2000-03-01T00:00:00.000000000",
                    "2000-04-01T00:00:00.000000000",
                    "2001-01-01T00:00:00.000000000",
                ],
                dtype="datetime64[ns]",
            ),
            dims=["time"],
            attrs={
                "axis": "T",
                "long_name": "time",
                "standard_name": "time",
                "bounds": "time_bnds",
            },
        ),
    }
)
ds["time_bnds"] = xr.DataArray(
    name="time_bnds",
    data=np.array(
        [
            ["2000-01-01T00:00:00.000000000", "2000-02-01T00:00:00.000000000"],
            ["2000-02-01T00:00:00.000000000", "2000-03-01T00:00:00.000000000"],
            ["2000-03-01T00:00:00.000000000", "2000-04-01T00:00:00.000000000"],
            ["2000-04-01T00:00:00.000000000", "2000-05-01T00:00:00.000000000"],
            ["2000-12-01T00:00:00.000000000", "2001-01-01T00:00:00.000000000"],
        ],
        dtype="datetime64[ns]",
    ),
    coords={"time": ds.time},
    dims=["time", "bnds"],
    attrs={"is_generated": "True"},
)

ds["ts"] = xr.DataArray(
    data=np.array([[[2]], [[np.nan]], [[1]], [[1]], [[2]]]),
    coords={"lat": ds.lat, "lon": ds.lon, "time": ds.time},
    dims=["time", "lat", "lon"],
)

In [3]:
ds

<xarray.Dataset>
Dimensions:    (lat: 1, lon: 1, time: 5, bnds: 2)
Coordinates:
  * lat        (lat) int64 -90
  * lon        (lon) int64 0
  * time       (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2001-01-01
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 2000-01-01 2000-02-01 ... 2001-01-01
    ts         (time, lat, lon) float64 2.0 nan 1.0 1.0 2.0

## 1. Set the TemporalAccessor class attributes to call internal methods.

In [4]:
ds.temporal._time_bounds = ds.time_bnds
ds.temporal._mode = "average"
ds.temporal._freq = ds.temporal._infer_freq()

## 2. Calculate the weights for monthly data (uses time bounds)

In [5]:
weights = ds.temporal._get_weights()
weights

<xarray.DataArray 'time_bnds' (time: 5)>
array([0.5, 1. , 1. , 1. , 0.5])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2001-01-01
    month    (time) int64 1 2 3 4 1

## 3. Calculate weighted means with generated weights.

In [6]:
# Grouped weighted average (annual cycle)
dv_gb_avg1 = (ds.ts * weights).groupby("time.month").sum()
dv_gb_avg1

<xarray.DataArray (month: 4, lat: 1, lon: 1)>
array([[[2.]],

       [[0.]],

       [[1.]],

       [[1.]]])
Coordinates:
  * lat      (lat) int64 -90
  * lon      (lon) int64 0
  * month    (month) int64 1 2 3 4

In [7]:
# Simple average with monthly weights
dv_avg1 = ds.ts.weighted(weights).mean(dim="time")
dv_avg1

<xarray.DataArray 'ts' (lat: 1, lon: 1)>
array([[1.33333333]])
Coordinates:
  * lat      (lat) int64 -90
  * lon      (lon) int64 0

## 4. Means with weights masked with `np.nan` for missing values

In [8]:
masked_weights = weights.copy()
masked_weights.data[1] = np.nan

In [9]:
dv_gb_avg2 = (ds.ts * masked_weights).groupby("time.month").sum()
dv_gb_avg2

<xarray.DataArray (month: 4, lat: 1, lon: 1)>
array([[[2.]],

       [[0.]],

       [[1.]],

       [[1.]]])
Coordinates:
  * lat      (lat) int64 -90
  * lon      (lon) int64 0
  * month    (month) int64 1 2 3 4

In [10]:
# ValueError: `weights` cannot contain missing values. Missing values can be replaced by `weights.fillna(0)`.
dv_avg2 = ds.ts.weighted(masked_weights).mean(dim="time")

In [ ]:
## 5. Means with weights masked with 0 for missing values
filled_weights = masked_weights.copy().fillna(0)

In [ ]:
dv_gb_avg3 = (ds.ts * filled_weights).groupby("time.month").sum()
dv_gb_avg3

In [ ]:
dv_avg3 = ds.ts.weighted(filled_weights).mean(dim="time")
dv_avg3

## Compare results

In [ ]:
# Generated weights vs. generated weights with np.nan for missing values
dv_gb_avg1.identical(dv_gb_avg2)

In [ ]:
# dv_avg1.identical(dv_avg2)  # Does not work since missing weight values must be filled

In [ ]:
# Generated weights vs. generated weights with 0 for missing values
dv_gb_avg1.identical(dv_gb_avg3)

In [ ]:
dv_avg1.identical(dv_avg3)

# Key Takeaways

- `weights.fillna(0)` is required if weights contain `np.nan` and the `.weighted().mean()` API is used for calculating simple weighted averages (e.g., ds.spatial.average())
  - `ValueError: 'weights' cannot contain missing values. Missing values can be replaced by ';weights.fillna(0)'.`
  - Question: Do the weights generated by our spatial averaging methods include `np.nan`?
- `weights` generated from time coordinates do not contain `np.nan` for missing values, so `weights.fillna(0)` is not required for temporal averaging

**In any case, multiplying any weight value (`np.nan`, 0, 1, etc.) with a missing value (represented in xarray as `np.nan`) results in `np.nan`. This has no affect on the final reduction operation. (Refer to below cells)**


In [ ]:
ds.ts * weights

In [ ]:
ds.ts * filled_weights

In [ ]:
ds.ts * masked_weights